In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:

pip install baostock -i https://pypi.tuna.tsinghua.edu.cn/simple/ --trusted-host pypi.tuna.tsinghua.edu.cn

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\acer\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
import baostock as bs
import pandas as pd

# login system
lg = bs.login()

# Get CSI 500 constituent stocks
rs = bs.query_zz500_stocks(date ='2021-01-01')

#Print the result set
zz500_stocks = []
while (rs.error_code == '0') & rs.next():
    # Get a record and merge the records together
    zz500_stocks.append(rs.get_row_data())
stocksdf = pd.DataFrame(zz500_stocks, columns=rs.fields)
# Output the result set to a csv file
stocksdf.to_csv("500_stocks.csv", encoding="gbk", index=False)
print(stocksdf)

# Log out of the system
bs.logout()

login success!
     updateDate       code code_name
0    2020-12-28  sh.600006      东风汽车
1    2020-12-28  sh.600008      首创股份
2    2020-12-28  sh.600021      上海电力
3    2020-12-28  sh.600022      山东钢铁
4    2020-12-28  sh.600026      中远海能
..          ...        ...       ...
495  2020-12-28  sz.300496      中科创达
496  2020-12-28  sz.300595      欧普康视
497  2020-12-28  sz.300618      寒锐钴业
498  2020-12-28  sz.300630      普利制药
499  2020-12-28  sz.300699      光威复材

[500 rows x 3 columns]
logout success!


In [4]:
codes = stocksdf['code'].values.tolist()
codenames = stocksdf['code_name'].values.tolist()

In [5]:
import baostock as bs
import pandas as pd

for i in codes:
    #### login system####
    lg = bs.login()

    #### Obtain historical K-line data of Shanghai and Shenzhen A shares####
    # For detailed indicator parameters, please refer to the "Historical Market Indicator Parameters" chapter; the "minute line" parameters are different from the "daily line" parameters. The "minute line" does not contain an index.
    # Minute line indicators: date, time, code, open, high, low, close, volume, amount, adjustflag
    # Weekly and monthly indicators: date, code, open, high, low, close, volume, amount, adjustflag, turn, pctChg
    rs = bs.query_history_k_data_plus(i,
        "date,time,code,open,high,low,close,volume,amount,adjustflag",
        start_date='2022-04-01', end_date='2022-07-31',
        frequency="30")


    #### Print result set####
    data_list = []
    while (rs.error_code == '0') & rs.next():
        # Get a record and merge the records together
        data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields)
    
    #### Output the result set to a csv file####   
    result.to_csv(i+".csv", index=False)
    print(i+" file saved")
print("Done for all 500 stocks")


#### Log out of the system####
bs.logout()

login success!
sh.600006 file saved
login success!
sh.600008 file saved
login success!
sh.600021 file saved
login success!
sh.600022 file saved
login success!
sh.600026 file saved
login success!
sh.600037 file saved
login success!
sh.600038 file saved
login success!
sh.600039 file saved
login success!
sh.600053 file saved
login success!
sh.600056 file saved
login success!
sh.600060 file saved
login success!
sh.600062 file saved
login success!
sh.600064 file saved
login success!
sh.600073 file saved
login success!
sh.600079 file saved
login success!
sh.600089 file saved
login success!
sh.600094 file saved
login success!
sh.600120 file saved
login success!
sh.600126 file saved
login success!
sh.600131 file saved
login success!
sh.600132 file saved
login success!
sh.600141 file saved
login success!
sh.600143 file saved
login success!
sh.600153 file saved
login success!
sh.600155 file saved
login success!
sh.600158 file saved
login success!
sh.600160 file saved
login success!
sh.600166 fil

In [27]:
in_cummulative_returns=[]
in_max_drawdowns=[]
in_samplesharpe_ratios=[]

In [28]:
import pandas as pd
import numpy as np

# Function to calculate Stochastic Oscillator
def calculate_stochastic_oscillator(df, k_window=14, d_window=3):
    low_min = df['low'].rolling(window=k_window).min()
    high_max = df['high'].rolling(window=k_window).max()
    k_percent = 100 * ((df['close'] - low_min) / (high_max - low_min))
    d_percent = k_percent.rolling(window=d_window).mean()
    return k_percent, d_percent

# Function to calculate Money Flow Index (MFI)
def calculate_mfi(df, window=14):
    typical_price = (df['high'] + df['low'] + df['close']) / 3
    money_flow = typical_price * df['volume']
    money_flow_positive = money_flow.where(typical_price > typical_price.shift(1), 0).rolling(window=window).sum()
    money_flow_negative = money_flow.where(typical_price < typical_price.shift(1), 0).rolling(window=window).sum()
    money_ratio = money_flow_positive / money_flow_negative
    mfi = 100 - (100 / (1 + money_ratio))
    return mfi

# Function to calculate Volume Weighted Average Price (VWAP)
def calculate_vwap(df):
    cum_volume = df['volume'].cumsum()
    cum_price_volume = (df['volume'] * (df['high'] + df['low'] + df['close']) / 3).cumsum()
    vwap = cum_price_volume / cum_volume
    return vwap

# Function to calculate Rate of Change (ROC)
def calculate_roc(df, window=14):
    roc = ((df['close'] - df['close'].shift(window)) / df['close'].shift(window)) * 100
    return roc

# Function to calculate Average True Range (ATR)
def calculate_atr(df, window=14):
    high_low = df['high'] - df['low']
    high_close_prev = abs(df['high'] - df['close'].shift(1))
    low_close_prev = abs(df['low'] - df['close'].shift(1))
    tr = pd.concat([high_low, high_close_prev, low_close_prev], axis=1).max(axis=1)
    atr = tr.rolling(window=window).mean()
    return atr

# Main function for mean-reverting strategy
def mean_reverting_strategy(df):
    # Calculate additional features
    k_percent, d_percent = calculate_stochastic_oscillator(df)
    mfi = calculate_mfi(df)
    vwap = calculate_vwap(df)
    roc = calculate_roc(df)
    atr = calculate_atr(df)
    
    # Calculate existing features
    df['RSI'] = calculate_rsi(df)
    upper_band, middle_band, lower_band = calculate_bollinger_bands(df)
    macd, signal = calculate_macd(df)
    
    # Generate signals
    df = generate_signals(df)
    
    # Backtest the strategy
    df = backtest_strategy(df)
    
    # Evaluate performance
    cumulative_return, max_drawdown, sharpe_ratio = evaluate_performance(df)
    
    return df, cumulative_return, max_drawdown, sharpe_ratio

# Function to calculate RSI
def calculate_rsi(df, window=14):
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to calculate Bollinger Bands
def calculate_bollinger_bands(df, window=20, num_std=2):
    rolling_mean = df['close'].rolling(window=window).mean()
    rolling_std = df['close'].rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    return upper_band, lower_band

# Function to generate signals
def generate_signals(df):
    df['Signal'] = 0
    df.loc[(df['close'] < df['Lower_Band']) & (df['RSI'] < 30), 'Signal'] = 1  # Oversold signal
    df.loc[(df['close'] > df['Upper_Band']) & (df['RSI'] > 70), 'Signal'] = -1  # Overbought signal
    return df

# Function for backtesting
def backtest_strategy(df):
    df['Position'] = 100 * df['Signal'].shift(1)  # Constant notional: hold a long position of 100 every day
    df['Returns'] = df['Position'] * df['close'].pct_change()
    return df

# Function for performance evaluation
def evaluate_performance(df):
    cumulative_returns = df['Returns'].cumsum()
    max_drawdown = (cumulative_returns - cumulative_returns.cummax()).min()
    sharpe_ratio = df['Returns'].mean() / df['Returns'].std() * np.sqrt(252)  # Annualized Sharpe Ratio
    return cumulative_returns.iloc[-1], max_drawdown, sharpe_ratio

# Main function for mean-reverting strategy
def mean_reverting_strategy(df):
    # Calculate features
    df['RSI'] = calculate_rsi(df)
    upper_band, lower_band = calculate_bollinger_bands(df)
    df['Upper_Band'] = upper_band
    df['Lower_Band'] = lower_band
    
    # Generate signals
    df = generate_signals(df)
    
    # Backtest the strategy
    df = backtest_strategy(df)
    
    # Evaluate performance
    cumulative_return, max_drawdown, sharpe_ratio = evaluate_performance(df)
    
    return df, cumulative_return, max_drawdown, sharpe_ratio

# Example usage
# Load historical stock price data into a DataFrame

for i in range(500):
    df = pd.read_csv(codes[i]+'.csv')
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)

    #Test the strategy from 2022-04-01 to 2022-06-30
    strategy_result, cumulative_return, max_drawdown, sharpe_ratio = mean_reverting_strategy(df['2022-04-01':'2022-06-30'])

    #Print performance metrics
    print("Cumulative Return:", cumulative_return)
    print("Max Drawdown:", max_drawdown)
    print("Sharpe Ratio:", sharpe_ratio)
    in_cummulative_returns.append(cumulative_return)
    in_max_drawdowns.append(max_drawdown)
    in_samplesharpe_ratios.append(sharpe_ratio)

Cumulative Return: 4.937562240801517
Max Drawdown: -4.997535846190405
Sharpe Ratio: 0.6606036878437431
Cumulative Return: 4.464534961502931
Max Drawdown: -0.7042253521126751
Sharpe Ratio: 1.3536454199873986
Cumulative Return: -2.0573496218283416
Max Drawdown: -5.885953632291074
Sharpe Ratio: -0.23772315334867827
Cumulative Return: 6.098455117907742
Max Drawdown: -1.1940404904476676
Sharpe Ratio: 1.73758733256412
Cumulative Return: 10.246107172850493
Max Drawdown: -4.722640576556515
Sharpe Ratio: 1.1774591467650641
Cumulative Return: 4.417660116200706
Max Drawdown: -3.6363636363636376
Sharpe Ratio: 0.5787102115436669
Cumulative Return: -4.197346677404523
Max Drawdown: -7.6341117144813175
Sharpe Ratio: -0.4302186414127692
Cumulative Return: -4.5023897339156465
Max Drawdown: -9.191177840539222
Sharpe Ratio: -0.5314065014533024
Cumulative Return: 10.523526604211542
Max Drawdown: -2.8657794404408525
Sharpe Ratio: 1.0686999230753396
Cumulative Return: 12.193828697577718
Max Drawdown: -2.3869

In [29]:
from statistics import mean
mean(in_samplesharpe_ratios),max(in_samplesharpe_ratios),min(in_samplesharpe_ratios)

(0.25048513556088103, 1.8921659959647241, -1.6111280491107955)

In [30]:
out_cummulative_returns=[]
out_max_drawdowns=[]
out_samplesharpe_ratios=[]

In [31]:
import pandas as pd
import numpy as np

# Function to calculate Stochastic Oscillator
def calculate_stochastic_oscillator(df, k_window=14, d_window=3):
    low_min = df['low'].rolling(window=k_window).min()
    high_max = df['high'].rolling(window=k_window).max()
    k_percent = 100 * ((df['close'] - low_min) / (high_max - low_min))
    d_percent = k_percent.rolling(window=d_window).mean()
    return k_percent, d_percent

# Function to calculate Money Flow Index (MFI)
def calculate_mfi(df, window=14):
    typical_price = (df['high'] + df['low'] + df['close']) / 3
    money_flow = typical_price * df['volume']
    money_flow_positive = money_flow.where(typical_price > typical_price.shift(1), 0).rolling(window=window).sum()
    money_flow_negative = money_flow.where(typical_price < typical_price.shift(1), 0).rolling(window=window).sum()
    money_ratio = money_flow_positive / money_flow_negative
    mfi = 100 - (100 / (1 + money_ratio))
    return mfi

# Function to calculate Volume Weighted Average Price (VWAP)
def calculate_vwap(df):
    cum_volume = df['volume'].cumsum()
    cum_price_volume = (df['volume'] * (df['high'] + df['low'] + df['close']) / 3).cumsum()
    vwap = cum_price_volume / cum_volume
    return vwap

# Function to calculate Rate of Change (ROC)
def calculate_roc(df, window=14):
    roc = ((df['close'] - df['close'].shift(window)) / df['close'].shift(window)) * 100
    return roc

# Function to calculate Average True Range (ATR)
def calculate_atr(df, window=14):
    high_low = df['high'] - df['low']
    high_close_prev = abs(df['high'] - df['close'].shift(1))
    low_close_prev = abs(df['low'] - df['close'].shift(1))
    tr = pd.concat([high_low, high_close_prev, low_close_prev], axis=1).max(axis=1)
    atr = tr.rolling(window=window).mean()
    return atr

# Main function for mean-reverting strategy
def mean_reverting_strategy(df):
    # Calculate additional features
    k_percent, d_percent = calculate_stochastic_oscillator(df)
    mfi = calculate_mfi(df)
    vwap = calculate_vwap(df)
    roc = calculate_roc(df)
    atr = calculate_atr(df)
    
    # Calculate existing features
    df['RSI'] = calculate_rsi(df)
    upper_band, middle_band, lower_band = calculate_bollinger_bands(df)
    macd, signal = calculate_macd(df)
    
    # Generate signals
    df = generate_signals(df)
    
    # Backtest the strategy
    df = backtest_strategy(df)
    
    # Evaluate performance
    cumulative_return, max_drawdown, sharpe_ratio = evaluate_performance(df)
    
    return df, cumulative_return, max_drawdown, sharpe_ratio

# Function to calculate RSI
def calculate_rsi(df, window=14):
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to calculate Bollinger Bands
def calculate_bollinger_bands(df, window=20, num_std=2):
    rolling_mean = df['close'].rolling(window=window).mean()
    rolling_std = df['close'].rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    return upper_band, lower_band

# Function to generate signals
def generate_signals(df):
    df['Signal'] = 0
    df.loc[(df['close'] < df['Lower_Band']) & (df['RSI'] < 30), 'Signal'] = 1  # Oversold signal
    df.loc[(df['close'] > df['Upper_Band']) & (df['RSI'] > 70), 'Signal'] = -1  # Overbought signal
    return df

# Function for backtesting
def backtest_strategy(df):
    df['Position'] = 100 * df['Signal'].shift(1)  # Constant notional: hold a long position of 100 every day
    df['Returns'] = df['Position'] * df['close'].pct_change()
    return df

# Function for performance evaluation
def evaluate_performance(df):
    cumulative_returns = df['Returns'].cumsum()
    max_drawdown = (cumulative_returns - cumulative_returns.cummax()).min()
    sharpe_ratio = df['Returns'].mean() / df['Returns'].std() * np.sqrt(252)  # Annualized Sharpe Ratio
    return cumulative_returns.iloc[-1], max_drawdown, sharpe_ratio

# Main function for mean-reverting strategy
def mean_reverting_strategy(df):
    # Calculate features
    df['RSI'] = calculate_rsi(df)
    upper_band, lower_band = calculate_bollinger_bands(df)
    df['Upper_Band'] = upper_band
    df['Lower_Band'] = lower_band
    
    # Generate signals
    df = generate_signals(df)
    
    # Backtest the strategy
    df = backtest_strategy(df)
    
    # Evaluate performance
    cumulative_return, max_drawdown, sharpe_ratio = evaluate_performance(df)
    
    return df, cumulative_return, max_drawdown, sharpe_ratio

# Example usage
# Load historical stock price data into a DataFrame

for i in range(44):
    df = pd.read_csv(codes[i]+'.csv')
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)

    #Test the strategy from 2022-04-01 to 2022-06-30
    strategy_result, cumulative_return, max_drawdown, sharpe_ratio = mean_reverting_strategy(df['2022-07-01':'2022-07-31'])

    #Print performance metrics
    print("Cumulative Return:", cumulative_return)
    print("Max Drawdown:", max_drawdown)
    print("Sharpe Ratio:", sharpe_ratio)
    out_cummulative_returns.append(cumulative_return)
    out_max_drawdowns.append(max_drawdown)
    out_samplesharpe_ratios.append(sharpe_ratio)


Cumulative Return: 0.39050490485035283
Max Drawdown: -1.9316940192340093
Sharpe Ratio: 0.22635013924979477
Cumulative Return: 1.029858832303998
Max Drawdown: -0.35211267605632646
Sharpe Ratio: 1.385986129197174
Cumulative Return: 0.4990122228186844
Max Drawdown: -1.5588370554869724
Sharpe Ratio: 0.3842408285048813
Cumulative Return: 0.5988023952095745
Max Drawdown: 0.0
Sharpe Ratio: 1.2284062992056615
Cumulative Return: 5.398336688951966
Max Drawdown: -0.42964554242748143
Sharpe Ratio: 2.2134065156429004
Cumulative Return: -1.5630849618471387
Max Drawdown: -3.987209838525796
Sharpe Ratio: -0.7239489025635848
Cumulative Return: -0.5346612996344202
Max Drawdown: -1.963484690831041
Sharpe Ratio: -0.35199992285013326
Cumulative Return: -1.0730737094156284
Max Drawdown: -1.5669344142912545
Sharpe Ratio: -1.1917394964318275
Cumulative Return: 1.194695625792952
Max Drawdown: -0.2328288707799775
Sharpe Ratio: 1.4106717520768397
Cumulative Return: -2.307687231396627
Max Drawdown: -4.06714009038

In [32]:
from statistics import mean
print(np.average(out_samplesharpe_ratios),max(out_samplesharpe_ratios),min(out_samplesharpe_ratios))

0.40874048226921017 2.4491515594860505 -1.860799904212739


In [37]:
mean(in_cummulative_returns),mean(in_max_drawdowns),max(in_cummulative_returns),max(in_max_drawdowns),min(in_cummulative_returns),min(in_max_drawdowns)

(1.4503136868301656,
 -4.956383615380759,
 22.48160344608832,
 -0.5494505494505475,
 -26.741505684158934,
 -26.741505684158934)

In [38]:
mean(out_cummulative_returns),mean(out_max_drawdowns),max(out_cummulative_returns),max(out_max_drawdowns),min(out_cummulative_returns),min(out_max_drawdowns)

(0.6751730041521232,
 -1.6187469352690804,
 5.398336688951966,
 0.0,
 -3.748298548866591,
 -6.2852625140153)

##RESULTS

Insample: 
    - Avg,Max,Min sharpe ratio:0.25 1.89 -1.61
    - Avg,Max,Min max drawdown:-4.95 -0.54 -26.74
    - Avg,Max,Min cummulative return:1.45 22.48 -26.74

Outsample:
    - Avg,Max,Min sharpe ratio:0.40 2.45 -1.86
    - Avg,Max,Min max drawdown:-1.61 0.0 -6.28
    - Avg,Max,Min  cummulative return:0.67 5.39 -3.74